# Attribution Case Study: Gemma - 2B

In this notebook we show how feature attribution (as calculated by our open source library!) can be used to understand model behaviour and help hypothesise reasons behind undesirable outputs. We showcase a case study using the Gemma-2B model and our gradient-based perturbation method described here. 

Since Gemma-2B is an open-source model, we will be downloading a local copy of it before calculating attribution values. To access Gemma-2B through huggingface, a huggingface account with approval is necessary. Please click [here](https://huggingface.co/google/gemma-2b) to go through the approval process. After approval, the huggingface account needs to be logged into using the CLI command `huggingface-cli login`.

We will use our `LocalLLMAttributor` function that employs a gradient-based perturbation technique to calculate attribution values.

## Importing libraries

In [1]:
import os
import sys
from pathlib import Path

from transformers import AutoModelForCausalLM, AutoTokenizer

sys.path.append(str(Path(os.getcwd()).parent))
from attribution.local_attribution import LocalLLMAttributor

/Users/jessicarumbelow/PIZZA/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading Gemma-2 model, tokenizer and embeddings

Make sure you've logged in using the `huggingface-cli login` command and have access to the Gemma-2B model hosted [here](https://huggingface.co/google/gemma-2b)

In [2]:
model_id = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto").cuda()
tokenizer = AutoTokenizer.from_pretrained(model_id)
embeddings = model.get_input_embeddings().weight.detach()

/Users/jessicarumbelow/PIZZA/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b-it.
401 Client Error. (Request ID: Root=1-667be01b-23f90ee5313034dc40ccbe17;70bfe96f-4f6d-4ee3-b2c5-01b2953b56fb)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must be authenticated to access it.

## Scenario 1: Apples to Apples

We ask Gemma-2B a simple question: 'Sam has 3 apples and Tom has 5 oranges. How many apples does Sam have?'. Gemma-2B correctly answers this question

In [ ]:
input_string = "Sam has 3 apples and Tom has 5 oranges. How many apples does Sam have?"
input_tokens = tokenizer(input_string, return_tensors="pt").input_ids.to(model.device)
output_tokens = model.generate(input_tokens, max_new_tokens=7).squeeze(0)
print(f"Output: {tokenizer.decode(output_tokens)}")

Output: <bos>Sam has 3 apples and Tom has 5 oranges. How many apples does Sam have?

The answer is 3.


## Scenario 2: Apples to Oranges?

We now slightly change the question to be: 'Sam has 3 apples and Tom has 5 oranges. How many oranges does Sam have?'. Gemma-2B gets this question wrong this time around, choosing to answer 5 oranges even though these oranges belong to Tom! 

In [ ]:
input_string = "Sam has 3 apples and Tom has 5 oranges. How many oranges does Sam have?"
input_tokens = tokenizer(input_string, return_tensors="pt").input_ids.to(model.device)
output_tokens = model.generate(input_tokens, max_new_tokens=7).squeeze(0)
print(f"Output: {tokenizer.decode(output_tokens)}")

Output: <bos>Sam has 3 apples and Tom has 5 oranges. How many oranges does Sam have?

The answer is 5.


## Observing Attribution values

Could we have predicted this behaviour if we observed the attribution values for Scenario 1? Let's take a look!

In [ ]:
attributor = LocalLLMAttributor(model=model, embeddings=embeddings, tokenizer=tokenizer)
attr_scores, token_ids = attributor.compute_attributions(
    input_string="Sam has 3 apples and John has 5 oranges. How many apples does Sam have?",
    generation_length=7,
)

attributor.print_attributions(
    word_list=tokenizer.convert_ids_to_tokens(token_ids),
    attr_scores=attr_scores,
    token_ids=token_ids,
    generation_length=7,
)

<bos>Sam▁has▁3▁apples▁and▁John▁has▁5▁oranges.▁How▁many▁apples▁does▁Sam▁have?

The▁answer▁is▁3.

                                                                                     
                                                                                      
                           The   ▁answer        ▁is          ▁          3          .  
 ──────────────────────────────────────────────────────────────────────────────────── 
  <bos>      0.7480   827.3525    2.4268   231.6353   551.3010   460.8656   488.2078  
  Sam        0.1883   185.5456    1.8554    38.9216   104.9274    24.2310   122.5083  
  ▁has       0.0620    57.6189    0.4312     8.8093    41.0717    38.5860    33.7193  
  ▁          0.0340    26.2205    0.2369     4.0255    24.5529    27.5611    18.0878  
  3          0.0337    28.9997    0.2823     5.0299    29.9640    59.1844    32.6388  
  ▁apples    0.0632    66.3430    0.4825     9.2379    43.1779    48.0342    40.1680  
  ▁and       0.0410    35.8075    0.3035     5.8184    28.4838    29.5762    19.1980  
  ▁John      0.0550    36.0833    0.4106     7.2349    24.1436    21.1167    19.9228  
  ▁has       0.0457    33.2646    0.3197     6.0864    27.5308    27.2985    20.2047  
  ▁          0.0342    18.9863    0.2035     3.3157    20.6291    17.4662    14.2829  
  5          0.0317    25.6511    0.2331     3.8841    17.9062    18.8697    20.8133  
  ▁oranges   0.0598    44.3548    0.3668     6.8322    33.3165    41.9764    27.7359  
  .          0.0435    32.0238    0.3410     6.2816    24.9733    17.1649    18.4730  
  ▁How       0.0459    36.6674    0.3877     7.0877    29.7497    22.6589    22.1241  
  ▁many      0.0470    36.2684    0.3264     6.4464    33.1295    30.9442    27.3642  
  ▁apples    0.0716    43.8999    0.3341     7.3740    46.5694    51.8812    32.5115  
  ▁does      0.0323    33.0140    0.2630     4.6643    23.4358    19.6655    14.9890  
  ▁Sam       0.0409    27.3865    0.3168     4.7084    28.0450    26.5090    17.9461  
  ▁have      0.0649    34.1126    0.2902     6.5851    35.5255    22.1672    23.9016  
  ?          0.1469    59.1601    0.5917    11.7581    68.4922    40.9595    41.5126  
             0.0000   150.9977    1.2861    19.3280    45.8290    32.2954    41.1756  
                                                                                      
                                                                                      
  The        0.0000     0.0000    1.2992     8.3744    20.3927     8.5602    16.5280  
  ▁answer    0.0000     0.0000    0.0000    27.1139    38.1941    18.5473    29.7542  
  ▁is        0.0000     0.0000    0.0000     0.0000    94.0511    27.8277    47.6029  
  ▁          0.0000     0.0000    0.0000     0.0000     0.0000    57.5610    14.0093  
  3          0.0000     0.0000    0.0000     0.0000     0.0000     0.0000    48.1393  
  .          0.0000     0.0000    0.0000     0.0000     0.0000     0.0000     0.0000

Looking at the attribution table, we see some sensible values associated with the output token `3` which is the correct answer:

|Input Token        | Output Token | Attribution |
|-------------------|--------------|-------------|
|`3`                | `3`          |  59.1844    |
|`apples` (First)   | `3`          |  48.0342    |
|`apples` (Second)  | `3`          |  51.8812    |
|`5`                | `3`          |  18.8697    |

There are high attributions for the input tokens that are strongly associated with the correct answer and low attribution for the input token `5` which would be the incorrect answer. There is something strange about some attribution values in this table for the output token `3`, notably the one associated with the`oranges` and both `Sam` input tokens.

|Input Token        | Output Token | Attribution |
|-------------------|--------------|-------------|
|`oranges`          | `3`          |  41.9764    |
|`Sam` (First)      | `3`          |  24.2310    |
|`Sam` (Second)     | `3`          |  26.5090    |

This value for the `oranges` token is surprisingly close to the input tokens associated with the correct answer input tokens and also quite a bit larger than the values associated with the incorrect answer input tokens. The values for both the `Sam` tokens however are surprisingly low and ideally should be higher since the model answers the question in Scenario 1 correctly.

Using the above analysis, we can come up with two hypotheses about model behaviour that could be potential causes for the failure in Scenario 2:

__Hypothesis 1__: Gemma-2B pays undue attention on the `oranges` tokens when asked about `apples` in Scenario 1 pointing to some confusion around which fruit is being asked about in the question.

__Hypothesis 2__: Gemma-2B doesn't associate `apples` strongly with `Sam` in Scenario 1 which causes the model to output the incorrect answer in Scenario 2

## Conclusion

In this notebook, we show a case study where attribution helps us generate hypotheses to understand model failures. There is much work to be done in this space and we encourage readers to further inspect more model failures and validate the usefulness of attribution. We welcome proposals for features and communication regarding experiments with our library to be sent to `hello@leap-labs.com`